**Author:** [Riccardo Guidotti](http://kdd.isti.cnr.it/people/riccardo-guidotti)  
**Python version:**  3.x

# Multivariate Time Series Classifiers

In [0]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [0]:
from tslearn.preprocessing import TimeSeriesScalerMinMax
from sklearn.model_selection import train_test_split, cross_val_score 

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [0]:
from sklearn.ensemble import RandomForestRegressor

In [0]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d %H:%M:%S')
df = pd.read_csv(r'C:\Users\loren\Desktop\DM II\dataset\datatraining.csv', header=0, index_col='date', date_parser = dateparse )
#df = df.drop(['Unnamed: 0'], axis = 1)
df.head()

C:\Users\loren\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.


,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
date,,,,,,
2015-02-04 17:51:00,23.18,27.2720,426.0,721.25,0.004793,1
2015-02-04 17:51:59,23.15,27.2675,429.5,714.00,0.004783,1
2015-02-04 17:53:00,23.15,27.2450,426.0,713.50,0.004779,1
2015-02-04 17:54:00,23.15,27.2000,426.0,708.25,0.004772,1
2015-02-04 17:55:00,23.10,27.2000,426.0,704.50,0.004757,1


In [0]:
class_name = 'Occupancy'
attributes = [col for col in df.columns if col != class_name]
X = df[attributes].values
y = df[class_name]

In [0]:
X_train = X[:int(X.shape[0]*0.7)]
X_test = X[int(X.shape[0]*0.7):]
y_train = y[:int(X.shape[0]*0.7)]
y_test = y[int(X.shape[0]*0.7):]

In [0]:
X.shape

(8143, 5)

In [0]:
np.linspace

<function numpy.linspace(start, stop, num=50, endpoint=True, retstep=False, dtype=None, axis=0)>

In [0]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)
i = 1
score = []
for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val = X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]
    for mf in np.linspace(1, 5, 6):
        for ne in np.linspace(50, 100, 6):
            for md in np.linspace(20, 40, 5):
                for msl in np.linspace(30, 100, 8):
                    rfr = RandomForestRegressor(
                        max_features=int(mf),
                        n_estimators=int(ne),
                        max_depth=int(md),
                        min_samples_leaf=int(msl))
                    rfr.fit(X_tr, y_tr)
                    score.append([i,
                                  mf, 
                                  ne,
                                  md, 
                                  msl, 
                                  rfr.score(X_val, y_val)])
    i += 1

In [0]:
scaler = TimeSeriesScalerMinMax()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
X_train.shape

(5700, 5, 1)

In [0]:
X_test.shape

(2443, 5, 1)

In [0]:
n_timesteps, n_outputs, n_features = X_train.shape[1], len(np.unique(y_train)), X_train.shape[2] 
print("TIMESTEPS: ", n_timesteps)
print("N. LABELS: ", n_outputs)
print("N. FEATURES: ", n_features)

TIMESTEPS:  5
N. LABELS:  2
N. FEATURES:  1


In [0]:
X_train_cnn, X_val_cnn, y_train_cnn, y_val_cnn = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train)

In [0]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Conv1D, Conv2D, MaxPool2D, Flatten, Dropout, LeakyReLU, GlobalAveragePooling1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

Using TensorFlow backend.
C:\Users\loren\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\loren\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\loren\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\loren\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [0]:
def build_lstm2(n_timesteps, n_outputs, n_features):
    model = Sequential()
    model.add(LSTM(4, input_shape=(n_timesteps, n_features), return_sequences=True, 
                        kernel_initializer='TruncatedNormal'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.3))
    
    #1
    for _ in range(2):
        model.add(LSTM(4, kernel_initializer='TruncatedNormal', return_sequences=True))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.04))   

    #2
    model.add(LSTM(32, kernel_initializer='TruncatedNormal', return_sequences=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.7))
    
    #3
    for _ in range(2):
        model.add(Dense(256, kernel_initializer='TruncatedNormal'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.2))
    #4
    for _ in range(1):
        model.add(Dense(64, kernel_initializer='TruncatedNormal'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.7))

    #5
    model.add(Dense(32, kernel_initializer='TruncatedNormal'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.4))
        
    model.add(Dense(n_outputs, activation='sigmoid'))
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [0]:
lstm2 = build_lstm2(n_timesteps, n_outputs, n_features)

In [0]:
lstm2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 5, 4)              96        
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 4)              16        
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 5, 4)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 4)              0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 5, 4)              144       
_________________________________________________________________
batch_normalization_2 (Batch (None, 5, 4)              16        
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 5, 4)             

In [0]:
rlr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)
mc = ModelCheckpoint('best_model_lstm2.h5', monitor='val_loss', save_best_only=True)

callbacks = [rlr, mc]

batch_size = 16
mini_batch_size = int(min(X_train.shape[0]/10, batch_size))

In [0]:
history_lstm2 = lstm2.fit(X_train_cnn, y_train_cnn, epochs=50, batch_size=mini_batch_size, callbacks=callbacks,
                          validation_data=(X_val_cnn, y_val_cnn)).history

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 4560 samples, validate on 1140 samples
Epoch 1/50
4560/4560 [==============================] - 6s 1ms/step - loss: 0.3709 - accuracy: 0.8193 - val_loss: 0.5243 - val_accuracy: 0.8000
Epoch 2/50
4560/4560 [==============================] - 3s 690us/step - loss: 0.2471 - accuracy: 0.8787 - val_loss: 0.7728 - val_accuracy: 0.8000
Epoch 3/50
4560/4560 [==============================] - 3s 688us/step - loss: 0.2372 - accuracy: 0.8846 - val_loss: 0.7366 - val_accuracy: 0.7974
Epoch 4/50
4560/4560 [==============================] - 3s 707us/step - loss: 0.2309 - accuracy: 0.8897 - val_loss: 0.2134 - val_accuracy: 0.8877
Epoch 5/50
4560/4560 [==============================] - 3s 704us/step - loss: 0.2268 - accuracy: 0.8844 - val_loss: 0.1893 - val_accuracy: 0.9061
Epoch 6/50
4560/4560 [==============================] - 3s 692us/step - loss: 0.2179 - accuracy: 0.8917 - val_loss: 0.2147 - val_

In [0]:
y_pred = np.argmax(lstm2.predict(X_test), axis=1)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.8158002455996726
F1-score [0.89151398 0.3902439 ]
              precision    recall  f1-score   support

           0       0.81      1.00      0.89      1855
           1       0.96      0.24      0.39       588

    accuracy                           0.82      2443
   macro avg       0.88      0.62      0.64      2443
weighted avg       0.84      0.82      0.77      2443



In [0]:
X_train_cnn.shape

(4560, 5, 1)

In [0]:
X_train_cnn2 = X_train_cnn.reshape(X_train_cnn.shape[0], X_train_cnn.shape[1], X_train_cnn.shape[2], 1)
X_val_cnn2 = X_val_cnn.reshape(X_val_cnn.shape[0], X_val_cnn.shape[1], X_val_cnn.shape[2], 1)
X_test_cnn2 = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

X_train_cnn2.shape

(4560, 5, 1, 1)

In [0]:
def build_cnn2(n_timesteps, n_features, n_outputs):
    input_shape = (n_timesteps, n_features, 1)

    model = Sequential()
    
    ks1_first = 3
    ks1_second = 3
    
    ks2_first = 4
    ks2_second = 4
    
    model.add(Conv2D(filters=(3), 
                     kernel_size=(ks1_first, ks1_second),
                     input_shape=input_shape, 
                     padding='same',
                     kernel_initializer='TruncatedNormal'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.02))
    
    for _ in range(2):
        model.add(Conv2D(filters=(4), 
                     kernel_size= (ks2_first, ks2_second), 
                         padding='same',
                     kernel_initializer='TruncatedNormal'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.2))  
    
    model.add(Flatten())
    
    for _ in range(4):
        model.add(Dense(64 , kernel_initializer='TruncatedNormal'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.4))
    
    for _ in range(3):
        model.add(Dense(128 , kernel_initializer='TruncatedNormal'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.3))
  
    model.add(Dense(1024 , kernel_initializer='TruncatedNormal'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.7))
        
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [0]:
cnn2 = build_cnn2(n_timesteps, n_features, n_outputs)

In [0]:
cnn2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 5, 1, 3)           30        
_________________________________________________________________
batch_normalization_9 (Batch (None, 5, 1, 3)           12        
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 5, 1, 3)           0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 5, 1, 3)           0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 1, 4)           196       
_________________________________________________________________
batch_normalization_10 (Batc (None, 5, 1, 4)           16        
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 5, 1, 4)          

In [0]:
rlr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)
mc = ModelCheckpoint('best_model_cnn2.h5', monitor='val_loss', save_best_only=True)

callbacks = [rlr, mc]

batch_size = 16
mini_batch_size = int(min(X_train.shape[0]/10, batch_size))

In [0]:
history_cnn2 = cnn2.fit(X_train_cnn2, y_train_cnn, epochs=50, batch_size=mini_batch_size, callbacks=callbacks,
                      validation_data=(X_val_cnn2, y_val_cnn)).history

Train on 4560 samples, validate on 1140 samples
Epoch 1/50
4560/4560 [==============================] - 5s 1ms/step - loss: 0.4151 - accuracy: 0.7928 - val_loss: 0.6778 - val_accuracy: 0.8000
Epoch 2/50
4560/4560 [==============================] - 3s 550us/step - loss: 0.2628 - accuracy: 0.8605 - val_loss: 0.6304 - val_accuracy: 0.9167
Epoch 3/50
4560/4560 [==============================] - 2s 543us/step - loss: 0.2370 - accuracy: 0.8662 - val_loss: 0.2048 - val_accuracy: 0.9228
Epoch 4/50
4560/4560 [==============================] - 2s 545us/step - loss: 0.2346 - accuracy: 0.8704 - val_loss: 0.2086 - val_accuracy: 0.9202
Epoch 5/50
4560/4560 [==============================] - 3s 550us/step - loss: 0.2291 - accuracy: 0.8814 - val_loss: 0.2010 - val_accuracy: 0.9307
Epoch 6/50
4560/4560 [==============================] - 3s 550us/step - loss: 0.2241 - accuracy: 0.8836 - val_loss: 0.2041 - val_accuracy: 0.9175
Epoch 7/50
4560/4560 [==============================] - 2s 543us/step - loss: 

In [0]:
y_pred = np.argmax(cnn2.predict(X_test_cnn2), axis=1)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.8837494883340156
F1-score [0.92885772 0.68232662]
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      1855
           1       1.00      0.52      0.68       588

    accuracy                           0.88      2443
   macro avg       0.93      0.76      0.81      2443
weighted avg       0.90      0.88      0.87      2443



In [0]:
def build_cnn3(n_timesteps, n_outputs, n_features):
    model = Sequential()
    
    model.add(Conv1D(filters=16, kernel_size=8, activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dropout(0.3))
    
    model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dropout(0.3))
    
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dropout(0.3))
    
    model.add(GlobalAveragePooling1D())
    
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [0]:
cnn3 = build_cnn3(n_timesteps, n_outputs, n_features)

ValueError: Negative dimension size caused by subtracting 8 from 5 for 'conv1d_1/convolution' (op: 'Conv2D') with input shapes: [?,1,5,1], [1,8,1,16].

In [0]:
cnn3.summary()

NameError: name 'cnn3' is not defined

In [0]:
rlr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)
mc = ModelCheckpoint('best_model_cnn2.h5', monitor='val_loss', save_best_only=True)

callbacks = [rlr, mc]

batch_size = 16
mini_batch_size = int(min(X_train.shape[0]/10, batch_size))

In [0]:
history_cnn3 = cnn3.fit(X_train_cnn, y_train_cnn, epochs=50, batch_size=mini_batch_size, callbacks=callbacks,
                      validation_data=(X_val_cnn, y_val_cnn)).history

In [0]:
y_pred = np.argmax(cnn3.predict(X_test), axis=1)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/

https://pyts.readthedocs.io/en/stable/generated/pyts.multivariate.classification.MultivariateClassifier.html#pyts.multivariate.classification.MultivariateClassifier